* LSTM (long short-term memory) is an improved structure to solve the problem of long-term dependencies

![img.png](https://miro.medium.com/max/1000/0*dhz-SOm-BW3rTEYa)

In [1]:
import numpy as np

from sklearn.metrics import accuracy_score
from tensorflow.keras.datasets import reuters
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Activation
from tensorflow.keras import optimizers
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import LSTM

In [2]:
num_words = 30000
maxlen = 50
test_split = 0.3

In [3]:
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words = num_words, maxlen = maxlen, test_split = test_split)

In [4]:
X_train[:5] , y_train[:5]

(array([list([1, 245, 273, 207, 156, 53, 74, 160, 26, 14, 46, 296, 26, 39, 74, 2979, 3554, 14, 46, 4689, 4329, 86, 61, 3499, 4795, 14, 61, 451, 4329, 17, 12]),
        list([1, 486, 341, 785, 26, 14, 482, 26, 255, 606, 252, 83, 146, 91, 102, 17, 12]),
        list([1, 245, 273, 397, 124, 53, 191, 26, 14, 83, 26, 39, 32, 32, 11, 14, 19, 12, 11, 88, 279, 59, 11, 14, 472, 59, 11, 17, 12]),
        list([1, 53, 745, 26, 14, 722, 26, 39, 7442, 18, 14, 6353, 18, 88, 258, 44, 11, 14, 119, 70, 11, 17, 12]),
        list([1, 486, 341, 119, 26, 14, 119, 26, 7, 255, 346, 606, 252, 83, 146, 91, 272, 17, 12])],
       dtype=object),
 array([3, 3, 3, 3, 3], dtype=int64))

In [5]:
# pad the sequences with zeros 
# padding parameter is set to 'post' => 0's are appended to end of sequences
X_train = pad_sequences(X_train, padding = 'post')
X_test = pad_sequences(X_test, padding = 'post')

In [6]:
X_train = np.array(X_train).reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1], 1))

In [7]:
X_train.shape

(1395, 49, 1)

In [8]:
y_data = np.concatenate((y_train, y_test))
y_data = to_categorical(y_data)

In [9]:
y_data[:5]

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 

In [10]:
y_train = y_data[:1395]
y_test = y_data[1395:]

In [11]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1395, 49, 1)
(599, 49, 1)
(1395, 46)
(599, 46)


In [12]:
def lstm():
    model = Sequential()
    model.add(LSTM(50, input_shape = (49,1), return_sequences = False))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [13]:

model = KerasClassifier(build_fn = lstm, epochs = 5, batch_size = 32, verbose = 1)

In [14]:
model.fit(X_train, y_train)

Train on 1395 samples
Epoch 1/5
1395/1395 [==============================] - 7s 5ms/sample - loss: 2.5151 - accuracy: 0.6452
Epoch 2/5
1395/1395 [==============================] - 0s 345us/sample - loss: 1.1971 - accuracy: 0.7147
Epoch 3/5
1395/1395 [==============================] - 1s 367us/sample - loss: 1.1291 - accuracy: 0.7147
Epoch 4/5
1395/1395 [==============================] - 1s 377us/sample - loss: 1.0942 - accuracy: 0.7147
Epoch 5/5
1395/1395 [==============================] - 1s 601us/sample - loss: 1.0076 - accuracy: 0.7441


In [15]:
y_pred = model.predict(X_test)

599/599 [==============================] - 1s 2ms/sample


In [18]:
y_test_ = np.argmax(y_test, axis = 1)

In [19]:
# accuracy improves by adopting LSTM structure
print(accuracy_score(y_pred, y_test_))

0.8163606010016694


### Stacked LSTM

![img.png](https://d3i71xaburhd42.cloudfront.net/b6c2a7b6e36e2a9f8c9eed7aa40d70d09d2c1706/4-Figure2-1.png)

In [20]:
def stacked_lstm():
    model = Sequential()
    model.add(LSTM(50, input_shape = (49,1), return_sequences = True))
    model.add(LSTM(50, return_sequences = False))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [26]:
model = KerasClassifier(build_fn = stacked_lstm, epochs = 10, batch_size = 32, verbose = 1)

In [27]:
model.fit(X_train, y_train)

Train on 1395 samples
Epoch 1/10
1395/1395 [==============================] - 7s 5ms/sample - loss: 2.1130 - accuracy: 0.6717
Epoch 2/10
1395/1395 [==============================] - 1s 863us/sample - loss: 1.1813 - accuracy: 0.7147
Epoch 3/10
1395/1395 [==============================] - 1s 840us/sample - loss: 1.1261 - accuracy: 0.7147
Epoch 4/10
1395/1395 [==============================] - 1s 941us/sample - loss: 1.0677 - accuracy: 0.7147
Epoch 5/10
1395/1395 [==============================] - 1s 872us/sample - loss: 1.1027 - accuracy: 0.7097 - loss: 1.1006 - accu
Epoch 6/10
1395/1395 [==============================] - 1s 709us/sample - loss: 1.0133 - accuracy: 0.7376
Epoch 7/10
1395/1395 [==============================] - 1s 719us/sample - loss: 0.8940 - accuracy: 0.7900
Epoch 8/10
1395/1395 [==============================] - 1s 725us/sample - loss: 0.8535 - accuracy: 0.7957
Epoch 9/10
1395/1395 [==============================] - 1s 762us/sample - loss: 0.8227 - accuracy: 0.8000
Epoc

In [28]:
y_pred = model.predict(X_test)

599/599 [==============================] - 2s 3ms/sample


In [29]:
print(accuracy_score(y_pred, y_test_))

0.8113522537562604
